In [ ]:
## This si also creating us and endpoint via gradio and its hosted on a Vertax AI workbench

In [2]:
import gradio as gr
import numpy as np
from PIL import Image, ImageOps

import time
start = time.time()

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from torchvision.models.segmentation import deeplabv3_resnet50
import torch
import torch.functional as F
import numpy as np
import requests
import torchvision
from PIL import Image
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image


image_url = "https://farm1.staticflickr.com/6/9606553_ccc7518589_z.jpg"
image = np.array(Image.open(requests.get(image_url, stream=True).raw))
rgb_img = np.float32(image) / 255
input_tensor = preprocess_image(rgb_img,
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
# Taken from the torchvision tutorial
# https://pytorch.org/vision/stable/auto_examples/plot_visualization_utils.html
model = deeplabv3_resnet50(pretrained=True, progress=False)
model = model.eval()

if torch.cuda.is_available():
    model = model.cuda()
    input_tensor = input_tensor.cuda()

output = model(input_tensor)
print(type(output), output.keys())
class SegmentationModelOutputWrapper(torch.nn.Module):
    def __init__(self, model): 
        super(SegmentationModelOutputWrapper, self).__init__()
        self.model = model
        
    def forward(self, x):
        return self.model(x)["out"]
    
model = SegmentationModelOutputWrapper(model)


# A simple mock function to mimic segmentation (thresholding for example)
def segment_image(image):
    # Convert to grayscale (PIL image)
    gray_image = ImageOps.grayscale(Image.fromarray(image))
    
    # Convert to NumPy array
    img_array = np.array(gray_image)
    rgb_img = np.float32(image) / 255
    input_tensor = preprocess_image(rgb_img,
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    output = model(input_tensor)
    normalized_masks = torch.nn.functional.softmax(output, dim=1).cpu()
    car_mask = normalized_masks[0, :, :, :].argmax(axis=0).detach().cpu().numpy()
    car_mask_uint8 = 255 * np.uint8(car_mask == 0)
    segmented_image = np.float32(car_mask == 0)
    
    # Apply a simple threshold to mimic segmentation
    #threshold_value = 128
    #segmentation_mask = img_array > threshold_value

    # Convert segmentation mask back to a PIL image
    #segmented_image = Image.fromarray(np.uint8(car_mask_float * 255))

    return segmented_image

# Create the Gradio interface
iface = gr.Interface(fn=segment_image, 
                     inputs="image", 
                     outputs="image", 
                     title="Image Segmentation",
                     description="Upload an image and see the segmentation result.")

# Launch the Gradio interface
iface.launch(share=True)


<class 'collections.OrderedDict'> odict_keys(['out', 'aux'])
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://a50bdbbf43f8a38ea5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File